## libs

In [152]:

import os
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import xlwings as xw #probelmy z pisaniem po excelu i api, ale do otwierania ok
import xlsxwriter as xls #tworzy nowe excele z logiką/ nie nadpisuje istniejących
# import openpyxl as ox #słaba funkcjonalność, ale pracuje na istniejących excelach
# from openpyxl.worksheet.table import Table, TableStyleInfo
import sqlite3 as sql

#from openpyxl import Workbook

## database - get data

In [2]:
db_path="D:/04_edu/04. Data/SQL_sample_database/"
db_file="northwind.db"

In [3]:
db_path+db_file

'D:/04_edu/04. Data/SQL_sample_database/northwind.db'

In [183]:
qry_1="""select  
strftime('%Y.%m', OrderDate) okres
,count(*) cnt
from   Orders
group by 1
order by 1"""

qry_2="""select  
strftime('%Y.%m', OrderDate) okres
,cast(strftime('%Y', OrderDate) as int) rok
,cast(strftime('%m', OrderDate) as int) okres
,count(*) cnt
from   Orders
group by 1,2,3
order by 1"""

qry_3="""select  
strftime('%Y.%m', OrderDate) okres
,cast(strftime('%Y', OrderDate) as int) rok
,cast(strftime('%m', OrderDate) as int) miesiac
,date(min(OrderDate)) minOrderDate
,max(OrderDate) maxOrderDate
,count(distinct a.OrderID) cnt_orders
,sum(Quantity) products_units
,ROUND(sum(Quantity*UnitPrice) ,2) value
from   Orders a
join [Order Details] b
on a.OrderID=b.OrderID
group by 1,2,3
order by 1"""


##
#weather.to_sql('df_name', conn,index=False, if_exists='replace')

#https://github.com/chineseballer06/Statistical-Analysis-of-Northwind-Database/blob/master/student.ipynb

In [184]:
conn = sql.connect(db_path+db_file)
df = pd.read_sql(qry_3, conn)
conn.close()
df

,okres,rok,miesiac,minOrderDate,maxOrderDate,cnt_orders,products_units,value
0,2016.07,2016,7,2016-07-04,2016-07-31,22,1462,30192.10
1,2016.08,2016,8,2016-08-01,2016-08-30,25,1322,26609.40
2,2016.09,2016,9,2016-09-02,2016-09-30,23,1124,27636.00
3,2016.10,2016,10,2016-10-01,2016-10-31,26,1738,41203.60
4,2016.11,2016,11,2016-11-01,2016-11-29,25,1735,49704.00
5,2016.12,2016,12,2016-12-02,2016-12-31,31,2200,50953.40
6,2017.01,2017,1,2017-01-01,2017-01-31,33,2401,66692.80
7,2017.02,2017,2,2017-02-03,2017-02-28,29,2132,41207.20
8,2017.03,2017,3,2017-03-03,2017-03-31,30,1770,39979.90
9,2017.04,2017,4,2017-04-01,2017-04-30,31,1912,55699.39


In [185]:
df.infer_objects().dtypes

okres              object
rok                 int64
miesiac             int64
minOrderDate       object
maxOrderDate       object
cnt_orders          int64
products_units      int64
value             float64
dtype: object

In [186]:
df.minOrderDate=df.minOrderDate.astype('string')

In [187]:
df

,okres,rok,miesiac,minOrderDate,maxOrderDate,cnt_orders,products_units,value
0,2016.07,2016,7,2016-07-04,2016-07-31,22,1462,30192.10
1,2016.08,2016,8,2016-08-01,2016-08-30,25,1322,26609.40
2,2016.09,2016,9,2016-09-02,2016-09-30,23,1124,27636.00
3,2016.10,2016,10,2016-10-01,2016-10-31,26,1738,41203.60
4,2016.11,2016,11,2016-11-01,2016-11-29,25,1735,49704.00
5,2016.12,2016,12,2016-12-02,2016-12-31,31,2200,50953.40
6,2017.01,2017,1,2017-01-01,2017-01-31,33,2401,66692.80
7,2017.02,2017,2,2017-02-03,2017-02-28,29,2132,41207.20
8,2017.03,2017,3,2017-03-03,2017-03-31,30,1770,39979.90
9,2017.04,2017,4,2017-04-01,2017-04-30,31,1912,55699.39


In [9]:
import locale
locale.localeconv()#["decimal_point"]


{'int_curr_symbol': '',
 'currency_symbol': '',
 'mon_decimal_point': '',
 'mon_thousands_sep': '',
 'mon_grouping': [],
 'positive_sign': '',
 'negative_sign': '',
 'int_frac_digits': 127,
 'frac_digits': 127,
 'p_cs_precedes': 127,
 'p_sep_by_space': 127,
 'n_cs_precedes': 127,
 'n_sep_by_space': 127,
 'p_sign_posn': 127,
 'n_sign_posn': 127,
 'decimal_point': '.',
 'thousands_sep': '',
 'grouping': []}

In [31]:
xw.__version__

'0.27.6'

## Excel path

In [129]:
dstr=datetime.now().strftime("%Y.%m.%d_%H.%M.%S")
#xls_path=Path("D:/04_edu/11_scoring/")
#xls_full_path=os.path.join(os.getcwd( ), xls_file)
xls_path=Path(os.getcwd( ))
xls_file="py_xls_"+dstr+".xlsx"
#xls_full_path=xls_path+xls_file
xls_full_path = xls_path / xls_file


In [130]:
xls_full_path

WindowsPath('D:/04_edu/11_scoring/py_xls_2023.04.09_18.03.03.xlsx')

In [134]:
os.path.isfile(xls_path /'py_excel_tes88.xlsx')

True

## xlsxwriter

https://xlsxwriter.readthedocs.io/contents.html

json to excel
https://stackoverflow.com/questions/65299781/convert-json-to-excel-by-python

https://xlsxwriter.readthedocs.io/example_pandas_table.html

https://xlsxwriter.readthedocs.io/working_with_tables.html

https://xlsxwriter.readthedocs.io/working_with_tables.html   kolumny jako formuła - definiowane w columns

https://xlsxwriter.readthedocs.io/working_with_cell_notation.html#cell-notation



nadpisywanie istniejącego arkusza openpyxl 
https://stackoverflow.com/questions/56148739/how-to-add-overwrite-a-sheet-of-an-already-existing-xlsx-file-without-changing-d

XLSWRITER zawsze tworzy nowy plik, nawet jak już istnieje to go nadpisze tworząc od zera

In [255]:
xls_file='py_xlsx_v1.xlsx'
display(str(os.path.isfile(xls_path /xls_file))+" <- Czy istniał taki plik?")

wb = xls.Workbook(xls_path /xls_file)
ws1=wb.add_worksheet('kalambur')
ws2=wb.add_worksheet('skoczek')
(max_row, max_col) = df.shape
ws1.add_table(0,0,max_row, max_col - 1, {'name': 'SalesData',
                                         'data': df.values,
                                        'columns': [{'header': nazwa} for nazwa in df.columns],
                                         'style': 'Table Style Light 8'
                                        })

df2 = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'Salary': [50000, 60000, 70000]})
c_shift=max_col+4
(max_row, max_col) = df2.shape

ws1.add_table(0,0+c_shift,max_row, c_shift+max_col - 1, {'name': 'inne_dane',
                                         'data': df2.values,
                                        'columns': [{'header': nazwa} for nazwa in df2.columns],
                                         'style': 'Table Style Light 8'
                                        })
#rozmiar tabeli 2:
t0=xls.utility.xl_cell_to_rowcol(ws1.tables[1].get('range').split(":")[0])
t1=xls.utility.xl_cell_to_rowcol(ws1.tables[1].get('range').split(":")[1])
rozmiar=tuple(np.subtract(t1, t0))


r0=xls.utility.xl_range(t0[0]+1, t0[1]+1, t1[0], t0[1]+1)
r1=xls.utility.xl_range(t0[0]+1, t0[1]+2, t1[0], t0[1]+2)

chart1 = wb.add_chart({'type': 'column'})
chart1.add_series({'name':'='+ws1.name+'!'+xls.utility.xl_rowcol_to_cell(t0[0], t0[1]+1),
                  'values': '='+ws1.name+'!'+r0
  
                  })
chart2 = wb.add_chart({'type': 'line'})
chart2.add_series({'name':'='+ws1.name+'!'+xls.utility.xl_rowcol_to_cell(t0[0], t0[1]+2),
                  'values': '='+ws1.name+'!'+r1,
                  'y2_axis': 1
                  })

chart1.combine(chart2)

chart1.set_title({  'name': 'Combine chart - secondary Y axis'})
chart1.set_x_axis({ 'name': 'tytułu osi X'})
chart1.set_y_axis({ 'name': 'oś 1: wiek'})

# Note: the y2 properties are on the secondary chart.
chart2.set_y2_axis({'name': 'oś 2: wynagrodzenie'})



# Insert the chart into the worksheet.
ws1.insert_chart('A7', chart1)


display('Rozmiar tabeli 2:'+str(list(rozmiar)))
display(ws1.dim_rowmax,display(ws1.dim_colmax))
ws1.autofit()
display(wb.worksheets())
wb.close()


'True <- Czy istniał taki plik?'

'Rozmiar tabeli 2:[3, 2]'

14

23

None

In [287]:
ws1.tables[1]

AttributeError: 'dict' object has no attribute 'data'

In [243]:
ws1.__dir__()
dir(ws1)

Help on int object:

class int(object)
 |  int([x]) -> integer
 |  int(x, base=10) -> integer
 |  
 |  Convert a number or string to an integer, or return 0 if no arguments
 |  are given.  If x is a number, return x.__int__().  For floating point
 |  numbers, this truncates towards zero.
 |  
 |  If x is not a number or if base is given, then x must be a string,
 |  bytes, or bytearray instance representing an integer literal in the
 |  given base.  The literal can be preceded by '+' or '-' and be surrounded
 |  by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
 |  Base 0 means to interpret the base from the string as an integer literal.
 |  >>> int('0b100', base=0)
 |  4
 |  
 |  Built-in subclasses:
 |      bool
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __ceil_

## Otwieranie i zamykanie z xlwings

In [256]:
wb = xw.Book(xls_path /xls_file)
# xwapp = xw.apps.active 

In [199]:
wb.save(xls_path /xls_file)
wb.app.quit()


## old xlwings

### konwersja typów w excelu

In [69]:
wb = xw.Book()

# Write the dataframe to a worksheet
ws = wb.sheets[0]
#ws.range('A1').value = df
ws.range('F1').options(pd.DataFrame, header=1, index=False, expand='table').value=df
# Infer the data type of each column and format the corresponding columns in the Excel data table
table_range = ws.range('F1').expand()
for i, col in enumerate(df.columns):
    if all(isinstance(val, int) for val in df[col] if not pd.isna(val)):
        table_range.columns[i].number_format = '0'
    elif all(isinstance(val, float) for val in df[col] if not pd.isna(val)):
        table_range.columns[i].number_format = '0,00'
#     elif all(isinstance(val, str) for val in df[col] if not pd.isna(val)):
#         table_range.columns[i].number_format = '@'
#     else:
#         table_range.columns[i].number_format = '@'
        #table_range.columns[i].number_format = 'General'

In [ ]:
#KOLUMNA WYLICZENIOWA
# Create a sample dataframe with two columns
df = pd.DataFrame({'Value 1': [10, 20, 30],
                   'Value 2': [5, 10, 15]})

# Open a new Excel workbook
wb = xw.Book()
sheet = wb.sheets[0]

# Convert the dataframe to an Excel table
table = sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value = df

# Add a calculated column to the Excel table
table.ListColumns.Add(SourceType=0, Formula='=[@[Value 1]] + [@[Value 2]]', Position=3, Name='Total')

# Format the calculated column as a number
sheet.range(table.ListColumns['Total'].DataBodyRange).number_format = '#,##0.00'

In [11]:
from datetime import datetime

In [18]:
xls_full_path

WindowsPath('D:/04_edu/11_scoring/py_xls_2023.04.08_19.59.33.xlsx')

In [19]:
#instancje excela otwarte
xw.apps.keys()

[]

In [56]:
#wb = xw.Book(xls_full_path)
xls_app = xw.App(visible=True)
wb = xls_app.books.open(xls_full_path)
xls_app.books[0].close()  #zamykam zeszyt 1 (pusty excel)
xls_app.books

Books([<Book [py_xls_learn_v01.xlsx]>])

In [15]:
wb.sheets.count
wb.sheets[0].tables

Tables([<Table 'gb_tab1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_tab3' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_NOWA' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, ...])

In [57]:
ark = wb.sheets[0]
ark['C23'].expand().clear() #czyści też tabelę
ark['C23'].value = [['a', 'b'], [1, 2]]
table = ark.tables.add(source=ark['C23'].expand(), name='gb_NOWA2')
ark.tables
#table

Tables([<Table 'gb_tab1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_tab3' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_NOWA' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, ...])

In [59]:
df=result.loc[result.rok>2016,:]
table_name='gb_Nowa_update'
if table_name in [table.name for table in ark.tables]:
    ark.tables[table_name].update(df, index=False)
else:
    ark.tables.add(source=ark['H30'], name=table_name).update(df, index=False)

In [53]:
# https://peltiertech.com/structured-referencing-excel-tables/
# https://support.microsoft.com/pl-pl/office/u%C5%BCywanie-odwo%C5%82a%C5%84-strukturalnych-w-tabelach-programu-excel-f5ed2452-2337-4f71-bed3-c8ae6d2b276e

In [18]:
xw.Range('gb_Nowa_update[okres2]').value

[7.0, 8.0, 9.0, 10.0, 11.0, 12.0]

In [20]:
xw.Range('gb_Nowa_update[[#Nagłówki]; [okres2]]').value

'okres2'

In [26]:
ark.range('gb_Nowa_update[[#Nagłówki]; [okres2]]').value

'okres2'

In [38]:
ark.range('gb_Nowa_update[[#Nagłówki];[#Dane]; [cnt_orders]:[products_units]]').value
ark.range('gb_Nowa_update[[#Dane]; [cnt_orders]:[products_units]]').value

[[22.0, 1462.0],
 [25.0, 1322.0],
 [23.0, 1124.0],
 [26.0, 1738.0],
 [25.0, 1735.0],
 [31.0, 2200.0]]

In [52]:
ark.range('gb_Nowa_update[cnt_orders];gb_Nowa_update[cnt_orders]').value

[22.0, 25.0, 23.0, 26.0, 25.0, 31.0]

In [12]:
ark['C20'].value = [['a', 'b'], [1, 2], [3, 4], [5, 6]]

In [73]:
ark['B2'].top

14.25

In [83]:
chart_top    = int(ark['N30'].top)  #kotwica
chart_left   = int(ark['N30'].left) #kotwica
chart_width  = int(ark['J1'].left)  #szerokość mierzona od A1
chart_height = int(ark['A20'].top)  #wysokość mierzona od A1
#list_length  = int(ws.range('D6').value)

In [89]:
chart = ark.charts.add()
chart.set_source_data(ark.range('gb_Nowa_update[cnt_orders]'))
chart.chart_type = 'line'
chart.top = chart_top
chart.left = chart_left
chart.width = chart_width
chart.height = chart_height
chart.name='gb_wykres'
chart.api[1].SetElement(2)  # Place chart title at the top
chart.api[1].ChartTitle.Text = 'Zamówenia'  # Change text of the chart title
chart.api[1].SeriesCollection(1).XValues = '=gb_Nowa_update[[rok]:[okres2]]'

In [90]:
ark.charts

Charts([<Chart 'Wykres 1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Chart 'gb_wykres' in <Sheet [py_xls_learn_v01.xlsx]ark1>>])

In [91]:
wb.save()
wb.close()
xls_app.quit()

In [4]:
list_data = [10, 20, 30, 20, 15, 30, 45]
df = pd.DataFrame(list_data,columns=['kol1'])
df

,kol1
0,10
1,20
2,30
3,20
4,15
5,30
6,45


In [83]:
wb = xw.Book(xls_full_path)
ws=wb.sheets['Test']
ws['A2'].value
xw.Range('Tabela_excela[ABC]').value #kolumna z table
ws.tables.api

<win32com.gen_py.Microsoft Excel 12.0 Object Library.ListObjects instance at 0x2295217220528>

In [41]:
wb['Test'].title
ws['A4'] = 4
ws.cell(row=4, column=2, value=10)
ws.cell(row=1, column=1).value=33
ws['A1'].value


33

## old openxl

In [40]:
if os.path.isfile(xls_full_path):
    wb = ox.load_workbook(xls_full_path)
else: 
    wb = ox.Workbook()
ws=wb.active
ws.title='Test'
ws['A1']='Moj pierwszy zapis'
ws.append(['drugi wiersz']) #dodaje na końcu arkusza
ws.append(['a','b','hlkhlk'])

ws['F1']='ABC'
tab = ox.worksheet.table.Table(displayName="Tabela_excela", ref="F1:F2")
# Add a default style with striped rows and banded columns
style = ox.worksheet.table.TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
tab.tableStyleInfo = style
ws.add_table(tab)

wb.save(xls_full_path)
wb.close()


writer = pd.ExcelWriter(xls_full_path,mode='a', engine='openpyxl'
                        #,if_sheet_exists="replace"
                        , if_sheet_exists="overlay" #istotne przy pracy na istniejących plikach
                       )
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=5, startcol=4)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=5)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=6)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=7)
df.to_excel(writer, sheet_name='Test' ,header=False, index=False, index_label=None, startrow=1, startcol=5)
writer.close()

wb = ox.load_workbook(xls_full_path)
ws=wb['Test']
ws.tables['Tabela_excela'].ref="F1:F8"
# ws.tables['MyTable'].ref = "A1:E5"
# print(len(ws['F']))

wb.save(xls_full_path)
wb.close()


In [71]:
wb = ox.load_workbook(xls_full_path)
ws=wb['Test']
t={key : value for key, value in ws.tables.items()}

wb.close()  

In [72]:
t

{'Tabela_excela': 'F1:F8'}

In [5]:
basepath = xls_path
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        print(entry)

gb_fx_data.ipynb
gb_fx_data_vars.parquet.gzip
gb_fx_data_vars.pckl
gb_scr.ipynb
gb_scr2.ipynb
hash.ipynb
ING_1_bucketing.ipynb
ING_2_feature_selection.ipynb
ING_3_skorecard_model.ipynb
ING_benchmarks_simple.ipynb
ING_benchmark_with_EBM.ipynb
ING_using-bucketing-process.ipynb
Midgard_schedules.ipynb
MM_hyperparams.ipynb
Od zera do MLa 4.ipynb
py_excel.ipynb
py_excel_tes88.xlsx
py_excel_test10.xlsx
py_excel_test9.xlsx
rsx_new.ipynb
Udemy PD Model COMPLETE- With Comments.ipynb
Udemy Preparation - COMPLETE.ipynb
Untitled.ipynb
utils.py
xxx_dataframe_plot.py
xxx_fstrings.py
_scorecard_pipe.ipynb
_scorecard_pipe_xgb.ipynb
__py_WIKI.ipynb
